In [ ]:
import random


class Machine:
    
    def __init__(self, vulnerable, networkValue, cost):
        self.vulnerable = vulnerable
        self.networkValue = networkValue
        self.cost = cost
        
    def changeVulnerable(self, vulnerable):
        self.vulnerable = vulnerable
    
    def changeCost(self, cost):
        self.cost = cost
        
    def pr(self):
        print(self.vulnerable, self.cost)

        
def printAllvalues(machineValues, vulnerableMachineValues, importantMachineValues):
    print('machine values = ', machineValues, 'vulnerable machine values = ', vulnerableMachineValues, 'important machine values =', importantMachineValues)

def GetMachineValues():
    machineValues = random.randint(1, 15)
    return machineValues

def GetVulnerableMachineValues(machineValues):
    vulnerableMachineValues = random.randint(1, machineValues)
    return vulnerableMachineValues

def GetImportantMachineValues(machineValues):
    importantMachineValues = random.randint(1, machineValues)
    return importantMachineValues

def GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues):
    #пока что сделаем цену cost фиксированной
    #дальше придумаем фукнцию для нее
    machineArray = []
    for i in range(machineValues):
        tmp = Machine(False, 10)
        machineArray.append(tmp)
    #здесь баг - может 2 и более раз сгенерироваться
    #одинаковое число и поэтому кол-во машин будет неточным
    for i in range(0, vulnerableMachineValues):
        j = random.randint(0, machineValues - 1)
        print('j = ', j)
        machineArray[j].changeVulnerable(True)
    return machineArray
#возвращает число уязвимых машин, так как из-за бага
#прошлое число уязвимых != новому
def GetVulnerableMachine(machineValues, machineArray):
    vulnerableMachine = 0
    for i in range(machineValues):
        if machineArray[i].vulnerable == True:
            vulnerableMachine += 1
    return vulnerableMachine

def CostFunction(s, a, Ei):
    return -(s * Ei *Ei)/(a * Ei - 1)

def UtilityDefender(s, a, Ei, machineValues):
    # l - процент понесенных потерь машин
    tmp = 0
    for i in range(machineValues):
        tmp += CostFunction(s, a, Ei)
    return - l - tmp

def UtilityAttacker(machineValues):
    #потом придумаем как считать l
    return l
    
def AttackerStrategyProportional():
    
    

#def AttackerStrategyHigh(mv):
    

#def DefenderStrategyNothing(mv):
    

#def DefenderStrategyProportional(mv):
    
    
#def DefenderStrategyProportional(mv):
    
    
#def UtilityAttacker():
    

#def UtilityDefender():
    
    
machineValues = GetMachineValues()
vulnerableMachineValues = GetVulnerableMachineValues(machineValues)
importantMachineValues = GetImportantMachineValues(machineValues)
machineArray = GetMachineArray(machineValues, vulnerableMachineValues, importantMachineValues)
printAllvalues(machineValues, vulnerableMachineValues, importantMachineValues)
for i in range(machineValues):
    machineArray[i].pr()
vulnerableMachine = GetVulnerableMachine(machineValues, machineArray)
print(vulnerableMachine)